<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment3/blob/main/DL_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [5]:
!curl https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar --output daksh.tar
!tar -xvf  'daksh.tar' 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0   204M      0  0:00:09  0:00:09 --:--:--  168M
dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script

In [7]:
def pre_process_data(path,input_tokenizer=None,target_tokenizer=None,input_length=None,target_length=None):
  input_texts = []
  target_texts = []
  
  df = pd.read_csv(path,sep="\t",names=["1", "2","3"]).astype(str)
  if input_tokenizer is None:
    # Shuffle rows in random order with a fixed seed(for reproducability)
    df=df.sample(frac=1,random_state=1)
  # Add all the  input and target texts with start sequence and end sequence added to target 
  for index, row in df.iterrows():
      input_text=row['2']
      target_text= row['1']
      # Skip empty lines/words
      if target_text =='</s>' or input_text=='</s>':
        continue
      
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)
  
  #only train set will have input_tokenizer as none. Validation and test will will use the same.
  if input_tokenizer is None:
    input_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', char_level=True)
    input_tokenizer.fit_on_texts(input_texts)
  input_tensor = input_tokenizer.texts_to_sequences(input_texts)
  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,padding='post')
  if target_tokenizer is None:
    target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', char_level=True)
    target_tokenizer.fit_on_texts(target_texts)
  #tokenize the text
  target_tensor = target_tokenizer.texts_to_sequences(target_texts)
  #pad the text
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,padding='post')
  #for dataset which is not training we pad to make maximum length same as train set.
  if input_length is not None and target_length is not None:
      input_tensor=tf.concat([input_tensor,tf.zeros((input_tensor.shape[0],input_length-input_tensor.shape[1]))],axis=1)
      target_tensor=tf.concat([target_tensor,tf.zeros((target_tensor.shape[0],target_length-target_tensor.shape[1]))],axis=1)
  return input_texts,input_tensor,input_tokenizer,target_texts,target_tensor,target_tokenizer

In [12]:
transliteration_target_language = 'kn'

In [13]:
input_texts,input_tensor,input_tokenizer,target_texts,target_tensor,target_tokenizer=data("/content/dakshina_dataset_v1.0/"+transliteration_target_language+"/lexicons/"+transliteration_target_language+".translit.sampled.train.tsv")
# val_input_texts,val_input_tensor,val_input_tokenizer,val_target_texts,val_target_tensor,val_target_tokenizer=data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv",input_tokenizer,target_tokenizer,input_tensor.shape[1],target_tensor.shape[1])
# test_input_texts,test_input_tensor,test_input_tokenizer,test_target_texts,test_target_tensor,test_target_tokenizer=data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv",input_tokenizer,target_tokenizer,input_tensor.shape[1],target_tensor.shape[1])

In [21]:
print(input_texts[:10])
print(input_tensor[:10])
print(target_texts[:10])
print(target_tensor[:10])
print(len(input_texts))
print(input_tensor.shape)
print(len(target_texts))
print(target_tensor.shape)

['jagala', 'nivruttaradaru', 'no', 'kodugegalu', 'harivannu', 'diipagalannu', 'gamanisabeku', 'baadhyathe', 'phalavantikeya', 'ananda']
[[21  1 12  1 10  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 3  2 13  4  7  6  6  1  4  1  5  1  4  7  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 3 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [14 16  5  7 12  9 12  1 10  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 8  1  4  2 13  1  3  3  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 5  2  2 18  1 12  1 10  1  3  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [12  1 17  1  3  2 11  1 19  9 14  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [19  1  1  5  8 15  1  6  8  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [18  8  1 10  1 13  1  3  6  2 14  9 15  1  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 1  3  1  3  5  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]
['\tಜಗಳ\n', '\tನಿವೃತ್ತರಾದರು\n', '